In [ ]:
import numpy as np
from helper import *
from cleaning import clean_tweet
import nltk

In [ ]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

In [3]:
print('Indexing word vectors.')

embeddings_index = {}
DIM_SIZE = 25
f = open('../Data/glove.twitter.27B.25d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 1193515 word vectors.


In [52]:
pos_train , neg_train = load_data()

In [53]:
len(pos_train.Text)

97889

In [54]:
clean_tweet("ah")

'ah'

In [55]:
def Tweet_to_GloVe(tweet):
    cleaned_tweet = clean_tweet(tweet) 
    tweet_list = cleaned_tweet.split()
    tweet_list_filtered =  list(set(tweet_list) & set(embeddings_index.keys()))  #remove words not in embedding 
    list_of_embeddings = np.array([embeddings_index[word] for word in tweet_list_filtered],dtype='float32')
    return np.mean(list_of_embeddings, axis = 0)

In [56]:
Tweet_to_GloVe(pos_train.Text[0])

array([ 0.43480918,  0.58307546,  0.0421031 , -0.0365528 , -0.24439183,
       -0.57389265,  1.2316928 ,  0.5250021 , -0.5268    ,  0.03623318,
       -0.42029274,  0.10034092, -3.9297547 , -0.3755418 , -0.26148337,
        0.24946953,  0.03143083, -0.5478605 , -0.34317997, -0.34596437,
        0.16184999, -0.18437268, -0.28717262,  0.61893445, -0.6549    ],
      dtype=float32)

In [ ]:
x = np.zeros((len(pos_train.Text), DIM_SIZE))
i = 0
for  tweet in pos_train.Text:
    x[i] = Tweet_to_GloVe(tweet)
    i += 1

In [ ]:
DIM_SIZE = 25